In [1]:
import pandas as pd

In [2]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Fusionner les données des films et des évaluations
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Créer une matrice utilisateur-film où les lignes sont les utilisateurs, les colonnes sont les films, et les valeurs sont les notations
user_movie_matrix = movie_ratings.pivot_table(index='userId', columns='movieId', values='rating')

# Remplacer les valeurs manquantes par 0
user_movie_matrix = user_movie_matrix.fillna(0)

# Calculer la similarité entre utilisateurs basée sur la corrélation de Pearson
user_similarity = user_movie_matrix.corr(method='pearson', min_periods=5)

In [3]:
def get_user_recommendations(user_id):
    # Obtenir les notations de l'utilisateur donné
    user_ratings = user_movie_matrix.loc[user_id]
    
    # Filtrer les films déjà notés par l'utilisateur
    unrated_movies = user_ratings[user_ratings.isnull()]
    
    # Calculer les recommandations en utilisant la corrélation de Pearson
    user_corr = user_similarity[user_id]  # Corrélations de l'utilisateur avec les autres utilisateurs
    recommended_movies = user_movie_matrix.apply(lambda row: row * user_corr)  # Poids basés sur la corrélation
    recommended_movies = recommended_movies.sum(axis=1)  # Somme pondérée des notations
    
    # Trier les films recommandés par note moyenne
    top_recommendations = recommended_movies.sort_values(ascending=False)
    
    # Exclure les films déjà notés par l'utilisateur
    top_recommendations = top_recommendations[~top_recommendations.index.isin(user_ratings.dropna().index)]
    
    # Obtenir les 10 meilleurs films recommandés
    top_10_movies = top_recommendations.head(10)
    
    # Obtenir les titres des films recommandés à partir du DataFrame 'movies'
    recommended_movie_titles = movies.loc[top_10_movies.index]['title']
    
    return recommended_movie_titles

In [12]:
user_id = 504
recommendations_with_titles = get_user_recommendations(user_id)
print("Les 10 meilleurs films recommandés avec titres pour l'utilisateur avec l'ID", user_id, ":")
print(recommendations_with_titles)

Les 10 meilleurs films recommandés avec titres pour l'utilisateur avec l'ID 504 :
607                          Striptease (1996)
559                           Space Jam (1996)
576                    Mulholland Falls (1996)
561    Some Folks Call It a Sling Blade (1993)
565                Operation Dumbo Drop (1995)
566                   Mrs. Winterbourne (1996)
570                   Feeling Minnesota (1996)
571                      Delta of Venus (1995)
572                               Angus (1995)
578                    Oliver & Company (1988)
Name: title, dtype: object
